In [2619]:
%run Preprocessing.ipynb

In [9]:
import warnings
warnings.filterwarnings('ignore')
import gurobipy as gp
import pandas as pd
import time

In [10]:
def Provide_h(x_v, k):
    G  = []
    for n in N:
        for v in range(0,n+1):
            G.append(2*v*h[n-1]*x_v[k,n,v])
    return G

In [11]:
def Provide_Q(x, x_v, k):
    Q = []
    for n in N:
        for v in range(0,n+1):
            Q.append(x[n,v]-x_v[k,n,v])
    return Q

## Level Method with ICC

### Approximation

In [12]:
def Approx(N, k, beta, x_v, contact, con, con_prob, M):
    m = gp.Model()
    m.setParam('OutputFlag', 0)

    x = {}
    for n in N:
        for v in range(0,n+1):
            x[n, v] = m.addVar(vtype='C', name="x[%s,%s]"%(n,v))
    z = m.addVar(vtype='C', name="z",lb = -1000)

    m.setObjective(z, gp.GRB.MINIMIZE) 

    m.update()
    for i in range(k+1):
        Q = Provide_Q(x, x_v, i)  # Define Q vector
        G = Provide_h(x_v, i)  # Create G matrix
        m.addConstr(gp.quicksum(gp.quicksum(v * h[n-1] * x_v[i, n, v]* x_v[i, n, v] for v in range(n+1)) for n in N) 
                    + gp.quicksum(G[j] * Q[j] for j in range(len(G))) <= z)

    m.update()

    for n in N:
        m.addConstr(gp.quicksum(x[n,v] for v in range(0, n+1)) == 1)
    b, e = 0.25, 0.83
    # Add the cut
    m.addConstr(1 + beta >= gp.quicksum(gp.quicksum(x[n,v]*(1/mu)*contact*h[n-1]*((1-b)
            *(n-v*e)+ b*(n-v*e)**2 + b*v*e*(1-e)) for v in range(0, n+1)) for n in N))
    for i in range(1, k+1):
        m.addConstr(gp.quicksum(con_prob[j] for j in M[i]) + beta >= gp.quicksum(gp.quicksum(gp.quicksum(x[n,v]*(1/mu)*con[j]*h[n-1]*((1-b)
            *(n-v*e)+ b*(n-v*e)**2 + b*v*e*(1-e)) for v in range(0, n+1)) for n in N) *con_prob[j] for j in M[i]) )
    
    m.update()
    m.optimize()
    
    #print(f"f hat: {m.objVal}")
    x_value = {}
    for n in N:
        for v in range(0,n+1):
            if x[n,v].X != 0:
                #print("x[%s,%s]="%(n,v), x[n,v].X)
                x_value[n,v] = x[n,v].X
                
            else:
                x_value[n,v] = 0

    z = m.objVal
    return z

## Projection

In [13]:
def Project(N, k, beta, z, best, x_v, contact, alpha, con, con_prob, M):
    m = gp.Model()
    m.setParam('OutputFlag', 0)

    x = {}
    for n in N:
        for v in range(0,n+1):
            x[n, v] = m.addVar(vtype='C', name="x[%s,%s]"%(n,v))
    
    m.setObjective(gp.quicksum(gp.quicksum(pow(x[n,v]-(x_v[k,n,v]-h[n-1]*v*x_v[k,n,v]),2) for v in range(0, n+1)) for n in N), gp.GRB.MINIMIZE)
    
    m.update()
    for i in range(k+1):
        G = Provide_h(x_v, i)  # Create G matrix
        Q = Provide_Q(x, x_v, i) # Define Q vector
        m.addConstr(gp.quicksum(gp.quicksum(v * h[n-1] * x_v[i, n, v]* x_v[i, n, v] for v in range(n+1)) for n in N) +gp.quicksum(G[j] * Q[j] for j in range(len(G))) 
                    <= (1-alpha)*z + alpha*best)
       
    for n in N:
        m.addConstr(gp.quicksum(x[n,v] for v in range(0, n+1)) == 1)
    
    b, e = 0.25, 0.83
    # Add the cut
    m.addConstr(1 + beta >= gp.quicksum(gp.quicksum(x[n,v]*(1/mu)*contact*h[n-1]*((1-b)
            *(n-v*e)+ b*(n-v*e)**2 + b*v*e*(1-e)) for v in range(0, n+1)) for n in N))
    for i in range(1, k+1):
        m.addConstr(gp.quicksum(con_prob[j] for j in M[i]) + beta >= gp.quicksum(gp.quicksum(gp.quicksum(x[n,v]*(1/mu)*con[j]*h[n-1]*((1-b)
            *(n-v*e)+ b*(n-v*e)**2 + b*v*e*(1-e)) for v in range(0, n+1)) for n in N) *con_prob[j] for j in M[i]) )
    m.update()
    
    m.optimize()
    #print(f"projection: {m.objVal}")
    x_value = {}
    for n in N:
        for v in range(0,n+1):
            if x[n,v].X > 0.01:
                #print("x[%s,%s]="%(n,v), x[n,v].X)
                x_value[k+1,n,v] = x[n,v].X            
            else:
                x_value[k+1,n,v] = 0
                
    return  x_value

In [14]:
# Initialization
m = [2.3, 2.0, 1.8, 1.2, 0.9, 0.6]
m_prob = [0.01, 0.1, 0.25, 0.25, 0.25, 0.14]
mean_contact = 0
for j in range(len(m)):
    mean_contact += m[j] * m_prob[j]
mu = 2.5
h = [0.41,0.33,0.18,0.45,0.06,0.02,0.01]

In [18]:
N = {1,2,3,4,5,6,7}
beta = 0.001
alpha = 0.3
x_v = {} # Initialization
x_v[0, 1, 1] = 1
x_v[0, 2, 2] = 1
x_v[0, 3, 3] = 1
x_v[0, 4, 4] = 1
x_v[0, 5, 5] = 1
x_v[0, 6, 6] = 1
x_v[0, 7, 7] = 1
for n in N:
    for v in range(0,n+1):
        if (0, n,v) not in x_v.keys():
            x_v[0, n, v] = 0
Good_so_far = []
M = {}

In [19]:
start_time = time.time()
for k in range(10): 
    
    print("--------------------k=%d----------------"%k)
    f_good_so_far = 0
    for n in N:
        for v in range(0,n+1):
            f_good_so_far += v * h[n-1] * x_v[k, n, v]* x_v[k, n, v]
    Good_so_far.append(f_good_so_far)   
#     print("Good_so_far = ", Good_so_far)
    best = min(Good_so_far)
    
#     print("-------Approximation------")
    z = Approx(N, k, beta, x_v, mean_contact,  m, m_prob, M)
    
#     print("-------Projection-------")
    x_value = Project(N, k, beta, z, best, x_v, mean_contact, alpha, m, m_prob, M)
    x_v.update(zip(x_value.keys(), x_value.values()))
    
    
    b, e = 0.25, 0.83
    total = 0
    T = []
    for i in range(len(m)):
        R_hv = 0
        for n in N:
            for v in range(n + 1):
                R_hv += x_v[k+1, n,v] * (1/mu)*m[i]*h[n-1]*((1-b)*(n-v*e)+b*(n-v*e)**2 + b*v*e*(1-e))
        total +=  max(0,R_hv-1)*m_prob[i] 
        if R_hv - 1 > 0:
            T.append(i)
    M[k+1] = T
    if total  <= beta :
        break
    
print("optimal value = ", best)
for n in N:
    for v in range(0,n+1):
        if x_v[k,n,v] > 0.01:
            print("x_v[",n,v, "]=" ,x_v[k,n,v])
print("Time =", time.time()-start_time)

--------------------k=0----------------
--------------------k=1----------------
--------------------k=2----------------
--------------------k=3----------------
optimal value =  1.5583889594037388
x_v[ 1 0 ]= 0.4682113582883071
x_v[ 1 1 ]= 0.5317886417116928
x_v[ 2 1 ]= 0.0665980167897309
x_v[ 2 2 ]= 0.933401983208854
x_v[ 3 2 ]= 0.07936502459038255
x_v[ 3 3 ]= 0.9206349753990037
x_v[ 4 3 ]= 0.5132471498547699
x_v[ 4 4 ]= 0.4867528488349771
x_v[ 5 3 ]= 0.1594069446150239
x_v[ 5 4 ]= 0.4466414288436662
x_v[ 5 5 ]= 0.39395162653387755
x_v[ 6 4 ]= 0.16831090411525745
x_v[ 6 5 ]= 0.40138032010223146
x_v[ 6 6 ]= 0.43030877566719467
x_v[ 7 5 ]= 0.13143302163365217
x_v[ 7 6 ]= 0.2754334982184533
x_v[ 7 7 ]= 0.5931334186521302
Time = 0.052288055419921875


## DEP formulation - Optimal Solution

In [22]:
def DEP(h, mu, N, beta,  con, con_prob):
    m = gp.Model()
    m.setParam('OutputFlag', 0)
    
    x, y = {}, {}
    for n in N:
        for v in range(0,n+1):
            x[n, v] = m.addVar(vtype='C', name="x[%s,%s]"%(n,v))
    for i in range(len(con)):
        y[i] = m.addVar(vtype='C', name="y[%s]"%(i))
        
    m.setObjective(gp.quicksum(gp.quicksum(v * h[n-1] * x[n,v]*x[n,v] for v in range(0, n+1)) for n in N), gp.GRB.MINIMIZE)
    
    b, e = 0.25, 0.83
    for i in range(len(con)):
        m.addConstr(1 + y[i] >= gp.quicksum(gp.quicksum((1/mu)*con[i]*h[n-1]*((1-b)
            *(n-v*e)+ b*(n-v*e)**2 + b*v*e*(1-e))
            *x[n,v] for v in range(0, n+1)) for n in N))    
  
    m.addConstr(gp.quicksum(con_prob[i] * y[i] for i in range(len(con))) <= beta)
    
    for n in N:
        m.addConstr(gp.quicksum(x[n,v] for v in range(0, n+1)) == 1)

    m.update()
    m.optimize()
    x_value = {}
    x_v = {}
    for n in N:
        for v in range(0,n+1):
            if x[n,v].X >0.1 :
                print("x[%s,%s]="%(n,v), x[n,v].X)
                x_value[n,v] = x[n,v].X
                x_v[n,v] = x[n,v].X
            else:
                x_value[n,v] = 0
    print(m.objVal)
    return x_v

In [23]:
start_time = time.time()
xx = DEP(h, mu, N, beta,  m, m_prob)
print("Time =", time.time()-start_time)

x[1,0]= 0.1270610375821679
x[1,1]= 0.8729389624178321
x[2,1]= 0.3509537417664538
x[2,2]= 0.6490462581507549
x[3,2]= 0.39573228239286695
x[3,3]= 0.6042677170562735
x[4,3]= 0.41492308484422774
x[4,4]= 0.5850769131967097
x[5,4]= 0.4074643289739546
x[5,5]= 0.5599191051807377
x[6,5]= 0.38731748614145883
x[6,6]= 0.5300875753294242
x[7,5]= 0.11444743039298094
x[7,6]= 0.37544077562844563
x[7,7]= 0.5101117837270118
1.943928888992441
Time = 0.00893092155456543
